# 什么是集成学习？

如果你随机向几千个人询问一个复杂问题，然后汇总它们的答案。在许多情况下你会发现，这个汇总的回答比专家的答案还要好，这被称为集体智慧，同样，如果你聚合一组预测器的预测，得到的预测结果也比最好的单个预测器要好，这样的一组预测器，我们称为集成，也被称为集成学习。而一个集成学习的算法则被称为集成方法。

虽然集成学习的具体算法和策略各不相同，但都共享同样的基本步骤，集成学习一般可分为以下三个步骤：
- (1) 找到误差互相独立的基分类器
- (2) 训练集分类器
- (3) 合并基分类器的结果

根据合并基分类器的方式分为Voting(投票)和Stacking(堆叠)两种，前者用投票的方式，将获得最多的结果作为最终的结果；后者用串行的方式，把前一个基分类器的结果输出到下一个分类器，将所有基分类器的输出结果相加(或者用更复杂的算法耦合)作为最终的输出

Voting代表算法有bagging和posting，Stacking代表算法Boosting，Boosting的典型算法有Adaboost和梯度提升

下面分别对上面提到的相关概念进行解释

# Bagging和Pasting

Bagging和Posting方法在训练过程中，各基分类器之间无强依赖，可以并行进行训练，其中很著名的算法之一是基于决策树基分类器的随机森林(random forest)。为了让基分类器之间相互独立，将训练集分为若干子集(当训练样本较少时，子集之间可能有重叠)。该方法更像是一个集体决策的过程，每个个体都进行单独学习，学习的内容可以相同，可以不同，也可以部分重叠，但由于个体之间存在差异，最终做出的判断不会完全一致，在最终做决策时，每个个体单独做出判断，再通过投票的方式做出最后的集体决策

Bagging和Posting的唯一区别在于对于同一个基分类器抽样的样本数据是否可以放回，bagging抽样后的数据可以放回继续抽样，但posting抽样后的数据不可以放回，换句话说，bagging和pasting都允许训练实例在多个预测器中被多次采样名单是只有bagging允许训练实例被同一个预测器多次采样

由于bagging和Posting是并行训练机制，所以可以通过不同的cpu内核甚至是不同的服务器并行地训练预测器和分类，这也正是bagging和pasting方法如此流行的原因之一，它们非常易于扩展

# Boosting

Boosting方法训练基分类器时采用串行的方式，各个基分类器之间有依赖

它的基本思想是将基分类器层层叠加，每一层在训练的时候，对前一层基分类器分错的样本，给与更高的权重，测试时，根据各层分类器的结果的加权得到最终的结果，这涉及到两组加权系数，将在Adaboost算法中详细说明

Boosting的过程很类似于人类学习的过程，我们学习新知识的过程往往是迭代的，第一遍学习的时候，我们会记住一部分知识，但往往也会犯一些错误，对于这些错误，我们的印象会很深，在第二遍学习的时候，就会针对犯过的错误的只是加强学习，以减少类似的错误发生，不断循环往复，直到犯错误的次数减少到很低的程度

## AdaBoost

AdaBoost算法：
- (1) 初始化每个实例权重$w^{(i)} = \frac{1}{m}$,m为样本个数
- (2) 使用第一个预测器训练，计算其误差率$r_1$ $$r_j = \frac{\sum_{i=1,j_{j}^{(i)}!=y^{(i)}}^mw^{i}}{\sum_{i=1}^mw^{(i)}}$$
- (3) 计算预测器的权重，预测器的准确率越高，其权重就越高，如果只是随机猜测，则其权重接近于零。但是，如果大部分情况下它都是错的(也就是准确率比随机猜测还低)，那么它的权重为负$$\alpha_j = \eta \log \frac{1-r_j}{r_j}$$,$\eta为学习率超参数，默认为1$
- (4) 对每个实例的权重进行更新$$w^{{i}} \leftarrow \begin{cases}
  w^{(i)}&(y_j^{(i)} == y^{(i)})\\
  w^{(i)}\exp(a_j)& (y_j^{(i)} != y^{(i)})
\end{cases}$$
- (5) 将所有实例的权重归一化(即除以$\sum_{i=1}^mw^{(i)}$)
- (6) 使用更新后的权重训练一个新的预测器，然后重复整个过程(计算新预测器的权重，更新实例权重，然后对另一个预测器进行训练)
- (7) 当到达所需数量的预测器或得到完美的预测器时，算法停止

预测的时候，AdaBoost就是简单地计算所有预测器的预测结果，并使用预测其权重对它们进行加权，最后，得到大多数加权投票的类别就是预测器给出的预测类别

## 梯度提升

另一个非常受欢迎的提升法是梯度提升(Gradient Boosting),更AdaBoost一样，梯度提升也是逐步在集成中添加预测器，每一个都对前序做出改正，不同之处在于，它不像Adaboost那样在每个迭代中调整实例权重，而是让新的预测器针对前一个预测器的残差进行拟合

举个简单的回归示例，使用决策树作为基础预测器(梯度提升当然也适用于回归任务)，这被称为梯度树提升或者梯度提升回归树(DBRT)

from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(x,y)

现在针对第一个预测器的残差，训练第二个DecisionTreeRegressor

y2 = y - tree_reg1.predict(x)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(x,y2)

然后，针对第二个预测器的残差，训练第三个回归器

y3 = y2 - tree_reg1.predict(x)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(x,y3)

现在，有了一个包含三棵树的集成，将所有树的预测相加，从而对新实例进行预测相加，从而对新实例进行预测
y_pred = sum(tree.predict(x_new) for tree in (tree_reg1,tree_reg2,tree_reg3))

# sklearn集成学习实践

## 训练随机森林分类器

在<机器学习之决策树原理和sklearn实践>博文中为卫星数据训练的决策树基础上训练一个随机森林分类器

需求:
- a.生产1000个训练集子集，每个子集包含随机挑选的100个实例，提示:使用sklearn的ShuffleSplit来实现
- b.使用之前得到的超参数，在每个子集上训练一个决策树，在测试集上评估这1000个决策树。因为训练集更小，所以这些决策树的表现可能比第一个决策树要差一点
- c.用每个测试集实例，生成1000个决策树的预测，然后仅保留次数最频繁的预测(可以使用Scipy的mode()函数)，这样你在测试集上获得大多数投票的结果
- d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高

### 代码实现

In [29]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split,ShuffleSplit

dataset = make_moons(n_samples=10000,noise=0.4)
features,labels = dataset[0],dataset[1]

x_train,x_test = train_test_split(features,test_size=0.2,random_state=42)
y_train,y_test = train_test_split(labels,test_size=0.2,random_state=42)

train_subset = []

# 创建1000个训练子集
ss = ShuffleSplit(n_splits=1000,test_size = 0.01,train_size=0.01,random_state=42)
for train_index,_ in ss.split(features):
    train_subset.append((features[train_index],labels[train_index]))
    

In [28]:
# 使用x_train和y_train训练一个决策时
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

decisionTree = DecisionTreeClassifier(criterion='gini')
param_grid = {'max_leaf_nodes': [i for i in range(2,10)]}
gridSearchCV = GridSearchCV(decisionTree,param_grid=param_grid,cv=3,verbose=1)
gridSearchCV.fit(x_train,y_train)
y_prab = gridSearchCV.predict(x_test)
print('accuracy score:',accuracy_score(y_test,y_prab))

print(gridSearchCV.best_params_)
print(gridSearchCV.best_estimator_)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
accuracy score: 0.8525
{'max_leaf_nodes': 9}
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=9, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.0s finished


In [37]:
# 克隆1000个决策树
from sklearn.base import clone
import numpy as np

estimators = [clone(gridSearchCV.best_estimator_) for _ in range(1000)]

#在测试集上评估这1000个决策树
accuracy_score_array = []
for tree, (x_mini_train, y_mini_train) in zip(estimators, train_subset):
    tree.fit(x_mini_train, y_mini_train)
    y_predict = tree.predict(x_test)
    accuracy_score_array.append(accuracy_score(y_test,y_predict))
print('avg accuracy_score:', np.mean(accuracy_score_array))

avg accuracy_score: 0.815643


In [41]:
# 用每个测试集实例，生成1000个决策树的预测，然后仅保留次数最频繁的预测
y_pred = np.zeros((len(estimators),len(x_test)),dtype=np.uint8)

for tree_index, tree in enumerate(estimators):
    y_pred[tree_index] = tree.predict(x_test)

In [42]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(y_pred, axis=0)

print(accuracy_score(y_test, y_pred_majority_votes.reshape([-1])))

0.8605


## RandomForestClassifier随机森林分类器

通过4.1已经发现，随机森林是决策树的集成，通常用bagging(有时也可能是pasting)方法训练，构成随机森林分类器有两种方法：
- （1）先构建一个BaggingClassifier然后将结果传输到DecisionTreeClassifier
    
    bag_clf = BaggingClassifier(
            DecisionTreeClassifier(splitter='random',max_leaf_nodes=16),
            n_estimators=500,max_samples=1.0,bootstrap=True,n_jobs=-1)
            
- (2) 使用RandomForestClassifier类，这种方法更方便，对决策树更优化(同样，对于回归任务也有一个RandomForestRegresson类)
    
    from sklearn.ensemble import RandomForestClassifier

    rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
    
    rnd_clf.fit(x_train,y_train)
    
    y_pred_rf = rnd_clf.predict(x_test)
    
RandomForestClassifier具有DecisionTreeClassifier的所有超参数，已经BaggingClassifier的所有超参数，前者用来控制树的生长，后者用来控制集成本身

随机树的生长引入了更多的随机性：<b>分裂节点时不再是搜索最好的特征，而是在一个随机生成的子集里搜索最好的特征，这导致决策树具有更大的随机性</b>，用更高的偏差换取更低的方差，总之，还是产生了一个整体性能更优的模型

### 重要特征

单看一颗决策树，重要的特征更可能出现在靠近根节点的位置，而不重要的特征通常出现在靠近叶节点的位置(甚至根本不出现)，因此，通过计算一个特征在森林中所有树上的平均深度，可以算出一个特征的重要程度

sklearn在训练结束后会自动计算每个特征的重要性，通过变量feature_importances_可以访问到这个结果

## ExtraTreesClassifier极端随机树

随机森林里单棵树的生长过程中，每个节点在分裂时仅考虑到了一个随机子集所包含的特征，如果我们对每个特征使用随机阈值，而 不是搜索得出的最佳阈值(如常规决策树),则可能让决策树生长得更加随机，这种极端随机得决策树组成的森林，被称为极端随机树集成。

同样，它也是以更高的偏差换取最低的方差，极端随机树训练起来比常规随机森林要快得多，因为<b>在每个节点上找到每个特征的最佳阈值是决策树生长中最耗时的任务之一</b>

> 通常来说，很难预先知道一个RandomForestClassifier是否比一个ExtraTreesClassifier更好或是更差，唯一的方法是两种都尝试一遍，然后使用交叉验证(还需要使用网格搜索调整超参数)进行比较

## VotingClassifier投票分类器

<b>投票分类器可以解决以下问题:</b>
如果你已经在完全相同的训练集上训练了5个不同的模型，并且它们都达到了95%的准确率，是否还可以通过结合这些模型来获得更好的结果？

<b>硬投票器和软投票器的区别：</b>硬投票器只是统计每个分类器的投票，然后挑选出票最多的类别。软投票分类器计算出每个类别的平均估算概率，然后选出概率最高的类别，它比硬投票器的表现更优，因为它给与那些高度自信的投票更高的权重

### VotingClassifier类参数说明

<ul>
    <li><b>estimators</b>: list of (string,estimator) tuples,eg.[('lr',logistic_clf),...]</li>
    <li><b>voting</b>: str,投票方式:{'hard','soft'}</li>
    <li><b>weights</b>: </li>
    <li><b>n_jobs</b>: </li>
    <li><b>flatten_transform</b>: bool,optional,default=None</li>
</ul>

### VotingClassifier属性说明

<ul>
    <li><b>estimators_</b>: list of classifiers</li>
    <li><b>named_estimators_</b>: </li>
    <li><b>classes_</b>: </li>
</ul>

### 示例

## bagging或pasting分类器

sklearn提供了一个简单的API，可用BaggingClassifier类进行bagging和pasting(或BaggingRegressor)

### BaggingClassifier类参数

<ul>
    <li><b>base_estimator</b>: object ot None,if None,then the base estimators is a decision tree</li>
    <li><b>n_estimators</b>: int,optional,default=10,zhe number of base estimators in ensemble</li>
    <li><b>max_samples</b>: int or float, optional (default=1.0),If int, then draw `max_samples` samples,If float, then draw `max_samples * X.shape[0]` samples</li>
    <li><b>max_features</b>: </li>
    <li><b>bootstrap</b>: defalult=True表示bagging,False表示Pasting</li>
    <li><b>bootstrap_features</b>: </li>
    <li><b>oob_score</b>: 是否在训练结束后自动进行包外评估，bool,optional,default=False</li>
    <li><b>warm_start</b>: </li>
    <li><b>n_jobs</b>: </li>
    <li><b>random_state</b>: </li>
    <li><b>verbose</b>: </li>
</ul>

### BaggingClassifier类属性

<ul>
    <li><b>base_estimator_</b>: </li>
    <li><b>estimators_</b>: </li>
    <li><b>estimators_samples_</b>: </li>
    <li><b>estimators_features_</b>: </li>
    <li><b>classes_</b>: </li>
    <li><b>n_classes_</b>: </li>
    <li><b>oob_score_</b>: 包外评估分数</li>
    <li><b>oob_decision_function_</b>: </li>
</ul>

### Random Patches和随机子空间

BaggingClassifier也支持对特征进行抽样，这通过两个超参数控制：max_features和bootstrap_features

它们的工作方式跟max_samples和boostrap相同，只是抽样对象不再是实例，而是特征,因此，每个预测器将用输入特征的随机子集进行训练

这对于处理高维输入(例如图像)特别有用，对训练实例和特征都可以进行抽样，被称为Random Patches方法，而保留所有训练实例(即bootstrap=False并且max_samples=1.0)但是对特征进行抽样(即bootstrap_features=True并且/或max_features<1.0),这被称为随机子空间法

对特征抽样给预测器带来更大的多样性，所以以略高一点的偏差换取了更低的方差

## AdaBoostClassifier

### AdaBoostClassifier类参数

<ul>
    <li><b>base_estimator</b>: </li>
    <li><b>n_estimators</b>: int,optianal,default=50</li>
    <li><b>learning_rate</b>: float,optional,default=1.</li>
    <li><b>algorithm</b>: {'SAMME','SAMME.R'},default='SAMME.R'</li>
    <li><b>random_state</b>: </li>
</ul>

### AdaBoostClassifier类属性

<ul>
    <li><b>estimators_</b>: </li>
    <li><b>classes_</b>: </li>
    <li><b>n_classes_</b>: </li>
    <li><b>estimator_weights_</b>: </li>
    <li><b>estimator_errors_</b>: </li>
    <li><b>feature_importances_</b>: </li>
</ul>

### 示例